## 네이버증권(https://finance.naver.com/research/) 리서치 리포트(pdf) 다운로드
- 6개 카테고리(시황정보, 투자정보, 종목분석, 산업분석, 경제분석, 채권분석)별로 폴더 생성
- 각 카테고리별로 '오늘 날짜' 기준 리포트(pdf) 파일 다운로드 및 저장

In [1]:
!pip install schedule
!pip install requests beautifulsoup4 selenium

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import random
import time
from urllib3.exceptions import InsecureRequestWarning

# SSL 경고 메시지 비활성화
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [12]:
## 해당 경로에 pdf 존재하는지 검사 추가
 ## 현재 문제점) 제목에 / 포함된 경우 경로 구분으로 인식해서 저장 안됨


def random_delay():
    delay = random.uniform(1, 3)  # 랜덤 시간
    time.sleep(delay)

def create_category_folders():
    categories = {
        '시황정보': 'https://finance.naver.com/research/market_info_list.naver',
        '투자정보': 'https://finance.naver.com/research/invest_list.naver',
        '종목분석': 'https://finance.naver.com/research/company_list.nhn',
        '산업분석': 'https://finance.naver.com/research/industry_list.naver',
        '경제분석': 'https://finance.naver.com/research/economy_list.naver',
        '채권분석': 'https://finance.naver.com/research/debenture_list.naver',
    }

    base_dir = "/Users/jykim/Documents/활동/일경험프로젝트형IT/dataPreprocessing/research_reports"
    
    ## 카테고리별 저장 폴더 생성
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
    
    for category in categories.keys():
        category_path = os.path.join(base_dir, category)
        if not os.path.exists(category_path):
            os.makedirs(category_path)
    
    return base_dir, categories

def get_pdf_link_and_data(columns, category):
    data = {}
    pdf_link = None
    
    if category in ['종목분석', '산업분석']:
        # 종목분석: 종목명(0), 제목(1), 증권사(2), 첨부(3), 작성일(4), 조회수(5)
        data = {
            'company': columns[0].text.strip(),
            'title': columns[1].text.strip(),
            'firm': columns[2].text.strip(),
            'date': columns[4].text.strip()
        }
        pdf_cell = columns[3]
    else:
        # 투자정보: 제목(0), 증권사(1), 첨부(2), 작성일(3), 조회수(4)
        data = {
            'title': columns[0].text.strip(),
            'firm': columns[1].text.strip(),
            'date': columns[3].text.strip()
        }
        pdf_cell = columns[2]
    
    # PDF 링크 찾기
    if pdf_cell.find('a'):
        pdf_link = pdf_cell.find('a').get('href')
    
    return data, pdf_link


def process_table_row(row, category, target_date):
    try:
        columns = row.find_all('td')
        if len(columns) >= 4:
            data, pdf_link = get_pdf_link_and_data(columns, category)
            
            if target_date in data['date']:
                if category in ['종목분석', '산업분석']:
                    filename = f"{data['company']}_{data['title']}_{data['firm']}_{data['date']}.pdf"
                else:
                    filename = f"{data['title']}_{data['firm']}_{data['date']}.pdf"
                
                if pdf_link:
                    if not pdf_link.startswith('http'):
                        pdf_link = 'https://finance.naver.com/' + pdf_link.lstrip('/')
                    return filename, pdf_link
    except Exception as e:
        print(f"Error processing row: {e}")
    return None, None

def download_research_reports():
    base_dir, category_urls = create_category_folders()
    target_date = "25.01.06"
    
    for category, url in category_urls.items():
        try:
            random_delay()
            response = requests.get(url, verify=False)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            table = soup.find('div', class_='box_type_m').find('table')
            rows = table.find_all('tr')
            
            for row in rows:
                filename, pdf_link = process_table_row(row, category, target_date)
                
                if filename and pdf_link:
                    category_path = os.path.join(base_dir, category)
                    filepath = os.path.join(category_path, filename)
                    
                    # 파일이 이미 존재하는지 확인
                    if os.path.exists(filepath):
                        print(f"File already exists: {filename} in {category}")
                        continue
                        
                    random_delay()
                    pdf_response = requests.get(pdf_link, verify=False)
                    if pdf_response.status_code == 200:
                        with open(filepath, 'wb') as f:
                            f.write(pdf_response.content)
                        print(f"Downloaded: {filename} to {category}")
                    
        except Exception as e:
            print(f"Error accessing {category} page: {e}")


# if __name__ == "__main__":
#     download_research_reports()


## 매 시간 정각, 자동 실행

In [14]:
import schedule
import time
from datetime import datetime

def job():
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"\n작업 시작 시간: {current_time}")
    download_research_reports()
    print(f"작업 완료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

def run_scheduler():
    # 매 시간 정각에 실행
    schedule.every().hour.at(":00").do(job)
    # schedule.every().day.at("09:30").do(job)  # 매일 오전 9시 30분
    
    # 프로그램 시작 시 즉시 한 번 실행
    job()
    
    print("스케줄러 시작(매 시간 정각에 데이터 수집)")
    print("프로그램을 종료 : Ctrl+C")
    
    try:
        while True:
            schedule.run_pending()
            time.sleep(1)
    except KeyboardInterrupt:
        print("\n===== 프로그램 종료")

if __name__ == "__main__":
    run_scheduler()



작업 시작 시간: 2025-01-06 04:01:07
작업 완료 시간: 2025-01-06 04:01:20

스케줄러가 시작되었습니다. 매 시간 정각에 데이터를 수집합니다.
프로그램을 종료하려면 Ctrl+C를 누르세요.

작업 시작 시간: 2025-01-06 04:01:20
작업 완료 시간: 2025-01-06 04:01:35


작업 시작 시간: 2025-01-06 04:01:35
작업 완료 시간: 2025-01-06 04:01:35


작업 시작 시간: 2025-01-06 05:00:00
작업 완료 시간: 2025-01-06 05:00:13


작업 시작 시간: 2025-01-06 05:00:13
작업 완료 시간: 2025-01-06 05:00:26


작업 시작 시간: 2025-01-06 05:00:26
작업 완료 시간: 2025-01-06 05:00:26


작업 시작 시간: 2025-01-06 05:00:26
작업 완료 시간: 2025-01-06 05:00:26


작업 시작 시간: 2025-01-06 05:00:26
작업 완료 시간: 2025-01-06 05:00:38


작업 시작 시간: 2025-01-06 06:00:00
작업 완료 시간: 2025-01-06 06:00:12


작업 시작 시간: 2025-01-06 06:00:12
작업 완료 시간: 2025-01-06 06:00:23


작업 시작 시간: 2025-01-06 06:00:23
작업 완료 시간: 2025-01-06 06:00:23


작업 시작 시간: 2025-01-06 06:00:23
작업 완료 시간: 2025-01-06 06:00:23


작업 시작 시간: 2025-01-06 06:00:23
작업 완료 시간: 2025-01-06 06:00:36


작업 시작 시간: 2025-01-06 07:00:00
작업 완료 시간: 2025-01-06 07:00:12


작업 시작 시간: 2025-01-06 07:00:12
작업 완료 시간: 2025-01-06 07:00:26


작업 시작 시